In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

## Prepare the fake data to evaluate

In [3]:
df = pd.DataFrame(
    [
        {
            "reference": "The Eiffel Tower is located in Paris, France. It was constructed in 1889 as the entrance arch to the 1889 World's Fair.",
            "query": "Where is the Eiffel Tower located?",
            "response": "The Eiffel Tower is located in Paris, France.",
        },
        {
            "reference": "The Great Wall of China is over 13,000 miles long. It was built over many centuries by various Chinese dynasties to protect against nomadic invasions.",
            "query": "How long is the Great Wall of China?",
            "response": "The Great Wall of China is approximately 13,171 miles (21,196 kilometers) long.",
        },
        {
            "reference": "The Amazon rainforest is the largest tropical rainforest in the world. It covers much of northwestern Brazil and extends into Colombia, Peru and other South American countries.",
            "query": "What is the largest tropical rainforest?",
            "response": "The Amazon rainforest is the largest tropical rainforest in the world. It is home to the largest number of plant and animal species in the world.",
        },
        {
            "reference": "Mount Everest is the highest mountain on Earth. It is located in the Mahalangur Himal sub-range of the Himalayas, straddling the border between Nepal and Tibet.",
            "query": "Which is the highest mountain on Earth?",
            "response": "Mount Everest, standing at 29,029 feet (8,848 meters), is the highest mountain on Earth.",
        },
        {
            "reference": "The Nile is the longest river in the world. It flows northward through northeastern Africa for approximately 6,650 km (4,132 miles) from its most distant source in Burundi to the Mediterranean Sea.",
            "query": "What is the longest river in the world?",
            "response": "The Nile River, at 6,650 kilometers (4,132 miles), is the longest river in the world.",
        },
        {
            "reference": "The Mona Lisa was painted by Leonardo da Vinci. It is considered an archetypal masterpiece of the Italian Renaissance and has been described as 'the best known, the most visited, the most written about, the most sung about, the most parodied work of art in the world'.",
            "query": "Who painted the Mona Lisa?",
            "response": "The Mona Lisa was painted by the Italian Renaissance artist Leonardo da Vinci.",
        },
        {
            "reference": "The human body has 206 bones. These bones provide structure, protect organs, anchor muscles, and store calcium.",
            "query": "How many bones are in the human body?",
            "response": "The adult human body typically has 256 bones.",
        },
        {
            "reference": "Jupiter is the largest planet in our solar system. It is a gas giant with a mass more than two and a half times that of all the other planets in the solar system combined.",
            "query": "Which planet is the largest in our solar system?",
            "response": "Jupiter is the largest planet in our solar system.",
        },
        {
            "reference": "William Shakespeare wrote 'Romeo and Juliet'. It is a tragedy about two young star-crossed lovers whose deaths ultimately reconcile their feuding families.",
            "query": "Who wrote 'Romeo and Juliet'?",
            "response": "The play 'Romeo and Juliet' was written by William Shakespeare.",
        },
        {
            "reference": "The first moon landing occurred in 1969. On July 20, 1969, American astronauts Neil Armstrong and Edwin 'Buzz' Aldrin became the first humans to land on the moon as part of the Apollo 11 mission.",
            "query": "When did the first moon landing occur?",
            "response": "The first moon landing took place on July 20, 1969.",
        },
    ]
)
df.head()

,reference,query,response
0,"The Eiffel Tower is located in Paris, France. ...",Where is the Eiffel Tower located?,"The Eiffel Tower is located in Paris, France."
1,"The Great Wall of China is over 13,000 miles l...",How long is the Great Wall of China?,"The Great Wall of China is approximately 13,17..."
2,The Amazon rainforest is the largest tropical ...,What is the largest tropical rainforest?,The Amazon rainforest is the largest tropical ...
3,Mount Everest is the highest mountain on Earth...,Which is the highest mountain on Earth?,"Mount Everest, standing at 29,029 feet (8,848 ..."
4,The Nile is the longest river in the world. It...,What is the longest river in the world?,"The Nile River, at 6,650 kilometers (4,132 mil..."


## Configure some environment variables

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    # "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [5]:
import nest_asyncio

from phoenix.evals import HallucinationEvaluator, VertexAIModel, QAEvaluator, run_evals

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
nest_asyncio.apply()  # This is needed for concurrency in notebook environments

In [7]:
# Set your VertextAI model
eval_model = VertexAIModel(project="boreal-dock-366706")

In [8]:
# Define your evaluators
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_evaluator = QAEvaluator(eval_model)

# We have to make some minor changes to our dataframe to use the column names expected by our evaluators
# for `hallucination_evaluator` the input df needs to have columns 'output', 'input', 'context'
# for `qa_evaluator` the input df needs to have columns 'output', 'input', 'reference'
df["context"] = df["reference"]
df.rename(columns={"query": "input", "response": "output"}, inplace=True)
assert all(
    column in df.columns for column in ["output", "input", "context", "reference"]
)

# Run the evaluators, each evaluator will return a dataframe with evaluation results
# We upload the evaluation results to Phoenix in the next step
hallucination_eval_df, qa_eval_df = run_evals(
    dataframe=df,
    evaluators=[hallucination_evaluator, qa_evaluator],
    provide_explanation=True,
)

run_evals |          | 0/20 (0.0%) | ⏳ 00:08<? | ?it/s 

Exception in worker on attempt 1: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 1: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 1: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 1: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 1: raised FailedPrecondition('Project `107495639396` 

run_evals |          | 0/20 (0.0%) | ⏳ 00:14<? | ?it/s 

Exception in worker on attempt 2: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 2: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 2: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 2: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 2: raised FailedPrecondition('Project `107495639396` 

run_evals |          | 0/20 (0.0%) | ⏳ 00:22<? | ?it/s 

Exception in worker on attempt 3: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 3: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 3: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 3: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 3: raised FailedPrecondition('Project `107495639396` 

run_evals |          | 0/20 (0.0%) | ⏳ 00:23<? | ?it/s 

Process was interrupted. The return value will be incomplete...


run_evals |          | 0/20 (0.0%) | ⏳ 00:27<? | ?it/s 

Exception in worker on attempt 4: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 4: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 4: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 4: raised FailedPrecondition('Project `107495639396` is not allowed to use Publisher Model `projects/boreal-dock-366706/locations/us-central1/publishers/google/models/text-bison@002`')
Requeuing...
Exception in worker on attempt 4: raised FailedPrecondition('Project `107495639396` 

In [1]:
from ragas import evaluate

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
evaluate

In [14]:
import os

from langchain_google_vertexai import VertexAI, VertexAIEmbeddings


In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    # "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json" # MacOS m4 pro
)

In [5]:
model = VertexAI(model_name="gemini-pro")



In [7]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


In [11]:
from datasets import Dataset

questions = ["What did the president say about Justice Breyer?",
             "What did the president say about Intel's CEO?",
             "What did the president say about gun violence?",
            ]
ground_truths = [["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
                ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
                ["The president asked Congress to pass proven measures to reduce gun violence."]]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(model.invoke(query))
  contexts.append([])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


In [15]:
result = evaluate(
    llm=model,
    dataset = dataset,
    embeddings=VertexAIEmbeddings(model="text-embedding-004"),
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating: 100%|██████████| 6/6 [00:12<00:00,  2.06s/it]


In [16]:
df

,user_input,retrieved_contexts,response,faithfulness,answer_relevancy
0,What did the president say about Justice Breyer?,[],"On January 26, 2022, President Biden announced...",1.0,0.7326
1,What did the president say about Intel's CEO?,[],"I'm sorry, but I'm not able to answer that que...",1.0,0.0000
2,What did the president say about gun violence?,[],I cannot answer that question because it refer...,0.8,0.0000
